In [10]:
import numpy as np
import pandas as pd
import matplotlib
import sklearn
import shap
import skopt
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor


In [17]:
N_JOBS = 6
errors = {}

def crossvalid(xx, yy, model, cvf):
    err_trn = []
    err_tes = []
    r_2_tes = []
    r_2_trn = []
    for train_index, test_index in cvf.split(xx):
        x_trn = pd.DataFrame(np.array(xx)[train_index], columns =xx.columns)
        x_tes = pd.DataFrame(np.array(xx)[test_index], columns = xx.columns)
        y_trn = np.array(yy)[train_index]
        y_tes = np.array(yy)[test_index]
        model.fit(x_trn,y_trn)
        x_trn_pred = model.predict(x_trn)
        x_tes_pred = model.predict(x_tes)
        err_tes.append(mean_squared_error(x_tes_pred, y_tes))
        err_trn.append(mean_squared_error(x_trn_pred, y_trn))
        r_2_tes.append(r2_score(y_tes, x_tes_pred))
        r_2_trn.append(r2_score(y_trn, x_trn_pred))
    v_tes = np.sqrt(np.array(err_tes))
    v_trn = np.sqrt(np.array(err_trn))
    print ("RMSE %1.3f (sd: %1.3f, min:%1.3f, max:%1.3f, det:%1.3f) ... train" % (v_trn.mean(), v_trn.std(),v_trn.min(), v_trn.max(),np.array(r_2_trn).mean()))
    print ("RMSE %1.3f (sd: %1.3f, min:%1.3f, max:%1.3f, det:%1.3f) ... test" % (v_tes.mean(), v_tes.std(), v_tes.min(), v_tes.max(), np.array(r_2_tes).mean()))
    ret = {}
    ret['trn_mean'] = v_trn.mean()
    ret['trn_std'] = v_trn.std()
    ret['trn_r2'] = np.array(r_2_trn).mean()
    ret['tes_mean'] = v_tes.mean()
    ret['tes_std'] = v_tes.std()
    ret['tes_r2'] = np.array(r_2_tes).mean()
    return ret


In [14]:
cvmodel = GridSearchCV(RandomForestRegressor(n_jobs = N_JOBS, random_state = 1126 ),
                                              param_grid = {'n_estimators':[100, 300, 500, 1000],
                                                            'max_depth': [8, 10]},
                                              n_jobs = -1)
                                
errors['RFR'] = corssvalid(feat, target, model, cvf)